In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("E:/!!Research/!!!Data/graph_analysis/lek_data/lek_data_binary_cleaned.csv")
df.head()

,Unnamed: 0,lek_id,year,activity,x_easting,y_northing
0,0,2,2007,True,672527,3730500
1,1,2,2008,True,672527,3730500
2,2,2,2009,True,672527,3730500
3,3,2,2010,True,672527,3730500
4,4,4,2007,True,669625,3729786


In [ ]:
# Group by 'site' and 'year', and aggregate by taking the max for 'activity'
df_result = df.groupby(['lek_id', 'year'], as_index=False).agg({'activity': 'max',
                                                                'x_easting' : 'first',
                                                                'y_northing' : 'first'})

df_result.head()